In [1]:
import torch as tr

print("CUDA: "+str(tr.cuda.is_available()))

print("GPUs: "+str(tr.cuda.device_count()))

CUDA: True
GPUs: 1


In [8]:
#syntax  会根据设备情况自动选择cpu或者gpu
device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
print(device)
a = tr.tensor([-10, 10, 10])
tr.exp(a) / tr.exp(a).sum()

cuda


tensor([1.0306e-09, 5.0000e-01, 5.0000e-01])

sentences = [
  "How are you",
  "Who are you",
  "Who are they",
  "Who are we",
  "Who am I",
  "Who am I",
  "Where are you going"
]

# replace to training dataset from training datas
## 1. all the elements in sentences are integers, elements equal to words in this model

## 2. read all elements from training data with randomized segments having same orders



In [9]:
path1 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem/data/first_training_set.csv"
path2 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem/data/second_training_set.csv"
path3 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem//data/third_training_set.csv"
path4 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem//data/fourth_training_set.csv"
path5 = "C:/Users/gzkei/PycharmProjects/cis667-secretary-problem//data/fifth_training_set.csv"


# pick 5000 (each 1000, imply by the parameter inside) segments in each dataset with randomized segments

sentences = []

import number_helpers

sen1 = number_helpers.augment_sentences(path1, 10, 100)
sen2 = number_helpers.augment_sentences(path2, 10, 100)
sen3 = number_helpers.augment_sentences(path3, 10, 100)
sen4 = number_helpers.augment_sentences(path4, 10, 100)
sen5 = number_helpers.augment_sentences(path5, 10, 100)

sentences.extend(sen1)
sentences.extend(sen2)
sentences.extend(sen3)
sentences.extend(sen4)
sentences.extend(sen5)

print(sentences[:5])
print('------------------------------------------------------------------------------------------------------------------------')
print('length:  '+str(len(sentences)))



[['3', '0', '3', '0', '2', '3', '2', '2', '3', '2', '0', '3', '0', '4', '2', '2', '0', '0', '2', '7', '1', '6', '2', '0', '1', '0', '3', '1', '1', '1', '4', '2', '4', '0', '5', '1', '2', '8', '0', '4', '1', '0', '3', '7', '0', '2', '4', '0', '3', '3', '1', '1', '6', '4', '1', '1', '0', '0', '2', '1', '4', '4', '2', '0', '0', '3', '0', '3', '3', '3', '0', '3', '5', '1', '3', '1', '4', '5', '3', '2', '0', '3', '5', '0', '0', '2', '1', '0', '5', '2', '0', '2', '0', '7', '0', '3', '2', '2', '6', '2', '-1'], ['0', '0', '3', '2', '3', '4', '3', '0', '2', '0', '2', '4', '2', '0', '1', '1', '4', '2', '1', '1', '3', '4', '1', '0', '5', '1', '1', '1', '2', '0', '3', '0', '0', '1', '5', '1', '0', '3', '2', '2', '5', '2', '1', '6', '7', '1', '0', '5', '6', '4', '3', '4', '0', '0', '3', '0', '4', '1', '0', '7', '5', '1', '1', '0', '1', '3', '8', '0', '1', '4', '1', '5', '4', '3', '4', '4', '0', '6', '4', '3', '0', '3', '1', '0', '2', '3', '1', '7', '0', '0', '3', '3', '0', '0', '0', '7', '7', '2', 

# start to set building

In [18]:

# Make a dictionary mapping each word to a one-hot tensor
words = set()
for sentence in sentences:
    for word in sentence:
        #for word in sentence.split(" "):
        words.add(word)
words = tuple(words)  # deterministic order

print(words[:5])

# PyTorch LSTM expects 3d tensors representing (sequence length, batch size, number of features)
I = tr.eye(len(words))


# 将数据放入gpu
I = I.to(device) 
I = I.cuda()

dictionary = {
    word: I[w].cuda().reshape(1, 1, len(words))
    for w, word in enumerate(words)}

print(len(dictionary))

('3', '4', '5', '0', '8')
11


# try to train the LSTM

In [19]:
# Define a small LSTM recurrent neural network with linear hidden-to-output layer
class Net(tr.nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        self.lstm = tr.nn.LSTM(input_size=len(words), hidden_size=hidden_size)
        self.readout = tr.nn.Linear(in_features=hidden_size, out_features=len(words))
        self.lstm.to(device)
        self.readout.to(device)

    def forward(self, x, v=None):
        _, v = self.lstm(x) if v is None else self.lstm(x, v)  # update hidden from input
        h, c = v  # LSTM hidden vector and internal so-called "cell state"
        y = self.readout(h)  # get output from hidden
        y = tr.softmax(y, dim=-1)  # make sure output is a probability distribution
        return y, v


print(Net(3))

Net(
  (lstm): LSTM(11, 3)
  (readout): Linear(in_features=3, out_features=11, bias=True)
)


# next

In [ ]:
net = Net(3)
# syntax 1
net.to(device)
net.cuda()
opt = tr.optim.SGD(net.parameters(), lr=0.001)
#opt = tr.optim.Adam(net.parameters(), lr=0.1)

for epoch in range(2000):

    batch_loss = 0.

    for sentence in sentences:
        #tokens = sentence.split(" ")
        tokens = sentence
        v = None  # no hidden activation at first time-step
        for t in range(len(tokens) - 1):
            y, v = net(dictionary[tokens[t]], v)
            y_target = dictionary[tokens[t + 1]]

            loss = tr.sum((y - y_target)**2) # MSE
            #loss = -tr.sum(y_target * tr.log(y))  # Cross-entropy
            batch_loss += loss

    batch_loss.backward()
    opt.step()
    opt.zero_grad()

    print(epoch, batch_loss.item())
    # if epoch % 100 == 0: print(epoch, batch_loss.item())

0 4562.1787109375
1 4484.845703125
2 4422.458984375
3 4371.78369140625
4 4331.923828125
5 4300.955078125
6 4276.25537109375
7 4255.92578125
8 4239.154296875
9 4225.75048828125
10 4215.53076171875
11 4207.9931640625
12 4202.533203125
13 4198.48779296875
14 4195.37890625
15 4192.90869140625
16 4190.85107421875
17 4189.11083984375
18 4187.6123046875
19 4186.30712890625
20 4185.154296875
21 4184.1357421875
22 4183.21826171875
23 4182.3974609375
24 4181.63037109375
25 4180.9541015625
26 4180.32763671875
27 4179.7509765625
28 4179.2265625
29 4178.7333984375
30 4178.27978515625
31 4177.85302734375
32 4177.4619140625
33 4177.09228515625
34 4176.740234375
35 4176.42724609375
36 4176.11865234375
37 4175.8310546875
38 4175.5517578125
39 4175.29833984375
40 4175.05029296875
41 4174.83251953125
42 4174.611328125
43 4174.3955078125
44 4174.19775390625
45 4174.009765625
46 4173.8369140625
47 4173.66015625
48 4173.49462890625
49 4173.33154296875
50 4173.19189453125
51 4173.04736328125
52 4172.90673828

413 4166.63330078125
414 4166.62890625
415 4166.6259765625
416 4166.623046875
417 4166.6181640625
418 4166.615234375
419 4166.61083984375
420 4166.607421875
421 4166.59912109375
422 4166.5986328125
423 4166.59326171875
424 4166.58251953125
425 4166.57958984375
426 4166.57763671875
427 4166.57763671875
428 4166.57373046875
429 4166.56640625
430 4166.5654296875
431 4166.56298828125
432 4166.55712890625
433 4166.5498046875
434 4166.54296875
435 4166.54443359375
436 4166.5390625
437 4166.5341796875
438 4166.52490234375
439 4166.521484375
440 4166.51708984375
441 4166.51220703125
442 4166.51123046875
443 4166.5029296875
444 4166.50048828125
445 4166.49365234375
446 4166.4853515625
447 4166.48681640625
448 4166.48193359375
449 4166.4794921875
450 4166.4765625
451 4166.47216796875
452 4166.470703125
453 4166.46142578125
454 4166.45458984375
455 4166.4482421875
456 4166.4501953125
457 4166.443359375
458 4166.44140625
459 4166.4384765625
460 4166.435546875
461 4166.42529296875
462 4166.42431640

# complete epoch
### and try prediction

In [8]:
# Try predicting
word = "3"
v = None
print(word)

for t in range(3):
  x = dictionary[word]
  y, v = net(dictionary[tokens[t]], v)
  y = y.squeeze() # ignore singleton dimensions for time-step/example
  w = y.argmax()
  word = words[w]
  prob = y[w]
  print(word, prob.item())

3
0 0.23600423336029053
1 0.23719550669193268
0 0.23819178342819214


In [13]:
# real prediction
current_sentence = ['3','5','7','4','3','2']
v = None
print(current_sentence)
for c in current_sentence:
    x = dictionary[c]
    
    print('x={x}'.format(x=x))
    print('----------------------------------------')

    y, v = net(dictionary[c], v)

    y = y.squeeze() # ignore singleton dimensions for time-step/example

    w = y.argmax()

    print('y={y}, v={v}, w={w}'.format(y=y,v=v,w=w))
    print('----------------------------------------')

    word = words[w]
    print('word={word} + w={w} ++ {words}'.format(word=word,w=w,words=words))
    print('----------------------------------------')
    prob = y[w]
    print(word, prob.item())
    print('----------------------------------------')

['3', '5', '7', '4', '3', '2']
x=tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])
----------------------------------------
y=tensor([0.0178, 0.0864, 0.0396, 0.0309, 0.1095, 0.1794, 0.1890, 0.0123, 0.0140,
        0.2781, 0.0430], grad_fn=<SqueezeBackward0>), v=(tensor([[[ 0.2002, -0.2487, -0.2157]]], grad_fn=<StackBackward0>), tensor([[[ 0.3453, -0.3728, -0.3543]]], grad_fn=<StackBackward0>)), w=9
----------------------------------------
word=0 + w=9 ++ ('8', '3', '6', '7', '4', '2', '1', '9', '-1', '0', '5')
----------------------------------------
0 0.2781238257884979
----------------------------------------
x=tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]])
----------------------------------------
y=tensor([0.0206, 0.0698, 0.0406, 0.0361, 0.1839, 0.2115, 0.1879, 0.0144, 0.0153,
        0.1761, 0.0439], grad_fn=<SqueezeBackward0>), v=(tensor([[[-0.0499, -0.3574,  0.1078]]], grad_fn=<StackBackward0>), tensor([[[-0.1329, -0.7479,  0.1404]]], grad_fn=<StackBackward0>)), 

In [15]:
import pickle
with open('beanlstm20211217.pkl', 'wb') as handle:
    pickle.dump(net, handle)  # like_people is a list with data
    input('Finish dumping')

Finish dumping
